In [1]:
import os
import sqlalchemy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
creds = pd.read_csv("sample_mimic_login_creds.csv")
myUserName = str(creds.iloc[0]['Username']).strip()
myPassword = str(creds.iloc[0]['password']).strip()

server_url = "mimic-db.renci.unc.edu"
database = "mimic"

# Create Connection String
conn_str = f"{myUserName}:{myPassword}@{server_url}/{database}"

# Create Engine
engine = sqlalchemy.create_engine('postgresql://' + conn_str)

In [3]:
query = """
-- First step
-- Pulls all the concept ids we need to care about
WITH icd10_conc AS (
SELECT * 
FROM omop.concept
WHERE concept_code LIKE 'C17.%' OR concept_code LIKE 'C18.%' OR
concept_code LIKE 'C21.%' OR  concept_code = 'Z86.010'
),

-- Second step 
-- Pulling the concepts
-- We want to pull the concept_id_2 for each of these
conc_maps AS (
SELECT concept_id_2
FROM omop.concept_relationship 
WHERE concept_id_1 in (SELECT concept_id FROM icd10_conc) 
AND relationship_id = 'Maps to'
),

-- third step -> ensuring all standard concepts
-- ensuring these are all standard concepts
stand_conc as (
SELECT concept_id_2 as cond_conc_id
FROM conc_maps cm
INNER JOIN omop.concept co ON cm.concept_id_2 = co.concept_id
WHERE co.standard_concept = 'S'
),

-- as of this point, the names are solid...

-- checking to see if they are in the condition occurrence table
canc_pats as (
SELECT DISTINCT co.person_id, co.condition_concept_id, co.condition_source_value
FROM omop.condition_occurrence co
WHERE co.condition_concept_id IN (SELECT * FROM stand_conc)
),

-- checking to see how many deaths we have records for
canc_deaths as (
SELECT * 
FROM omop.death od
WHERE od.person_id IN (SELECT person_id from canc_pats)
),

-- finding death dates
death_date as (
SELECT person_id, death_datetime
FROM omop.death od
WHERE od.person_id IN (SELECT person_id from canc_pats)
),

-- pulling max & min dates for patients -> range of records
-- earliest visit start to latest visit end provides us with visit dif
vis_max_min AS (
SELECT vo.person_id, MIN(vo.visit_start_datetime) AS first_vis, 
MAX(vo.visit_end_datetime) AS last_vis
FROM omop.visit_occurrence vo
WHERE vo.person_id IN (SELECT person_id FROM canc_pats)
GROUP BY vo.person_id
),

-- earliest procedure start to latest procedure end
proc_max_min AS (
SELECT po.person_id, MIN(po.procedure_datetime) AS first_proc , 
MAX(po.procedure_datetime) AS last_proc
FROM omop.procedure_occurrence po
WHERE po.person_id IN (SELECT person_id FROM canc_pats)
GROUP BY po.person_id
),
	
-- earliest drug exposure start to latest drug exposure end
drug_max_min AS (
SELECT de.person_id, MIN(drug_exposure_start_datetime) AS first_drug,
MAX(drug_exposure_end_datetime) AS last_drug
FROM omop.drug_exposure de
WHERE de.person_id IN (SELECT person_id FROM canc_pats)
GROUP BY de.person_id
),
	
-- earliest measurement to latest measurement
-- slows the running down though :(
meas_max_min AS (
SELECT om.person_id, MIN(om.measurement_datetime) AS first_meas, 
MAX(om.measurement_datetime) AS last_meas
FROM omop.measurement om
WHERE om.person_id IN (SELECT person_id FROM canc_pats)
GROUP BY om.person_id
),

-- no device exposures so we can just ignore that...

-- pulling the first and last datapoint date for each of the patients
num_years as (
SELECT cp.person_id, ((
	greatest(last_vis, last_proc, last_drug, last_meas)::date -
	least(first_vis, first_proc, first_drug, first_meas)::date)/365.25)
	 AS num_years	
FROM canc_pats cp
INNER JOIN vis_max_min vmm ON cp.person_id = vmm.person_id
INNER JOIN proc_max_min pmm ON cp.person_id = pmm.person_id
INNER JOIN drug_max_min dmm ON cp.person_id = dmm.person_id
INNER JOIN meas_max_min mmm ON cp.person_id = mmm.person_id
),

-- figuring out the start of the dx
dx_date AS (
SELECT person_id, MIN(condition_start_datetime) AS pat_dx_date
FROM omop.condition_occurrence co
WHERE co.person_id IN (SELECT person_id FROM canc_pats) AND
co.condition_concept_id IN (SELECT cond_conc_id FROM stand_conc)
GROUP BY co.person_id
)

-- working to make a table 1
-- pulling races, genders, and ethnicities for each patient, also age at dx
SELECT cp.person_id, c1.concept_name AS race, c2.concept_name AS gend, c3.concept_name AS ethn,
(dd.death_datetime - op.birth_datetime)/365.25 AS death_age, 
CASE WHEN cd.person_id is NULL THEN '0' ELSE '1' END AS pat_dec
FROM canc_pats cp
INNER JOIN omop.person op on OP.person_id = cp.person_id
INNER JOIN omop.concept c1 on c1.concept_id = op.race_concept_id -- finding the race
INNER JOIN omop.concept c2 on c2.concept_id = op.gender_concept_id -- finding gender
INNER JOIN omop.concept c3 on c3.concept_id = op.ethnicity_concept_id -- finding ethnicity
LEFT JOIN death_date as dd on cp.person_id = dd.person_id -- finding age at death (if we have record of death)
LEFT JOIN canc_deaths cd on cp.person_id = cd.person_id  -- finding deaths
"""

In [4]:
table1 = pd.read_sql_query(sqlalchemy.text(query), engine)

In [18]:
table1["race"].value_counts(normalize=False)

White                     154
African American           24
Unknown racial group       16
Other ethnic non-mixed      7
Asian                       5
Hispanic                    4
Chinese                     2
European                    1
Caribbean Island            1
Asian Indian                1
African                     1
Vietnamese                  1
Name: race, dtype: int64

In [8]:
table1["race"].value_counts(normalize=True)

White                     0.709677
African American          0.110599
Unknown racial group      0.073733
Other ethnic non-mixed    0.032258
Asian                     0.023041
Hispanic                  0.018433
Chinese                   0.009217
European                  0.004608
Caribbean Island          0.004608
Asian Indian              0.004608
African                   0.004608
Vietnamese                0.004608
Name: race, dtype: float64

In [ ]:
table1["gend"].value_counts(normalize=False)

In [10]:
table1["gend"].value_counts(normalize=True)

MALE      0.557604
FEMALE    0.442396
Name: gend, dtype: float64

In [19]:
table1["pat_dec"].value_counts(normalize=False)

1    124
0     93
Name: pat_dec, dtype: int64

In [15]:
table1["pat_dec"].value_counts(normalize=True)

1    0.571429
0    0.428571
Name: pat_dec, dtype: float64

In [17]:
table1["death_age"].mean()

Timedelta('113 days 03:14:21.266476734')